In [1]:
# Google Drive 마운트 확인
from google.colab import drive
drive.mount('/content/drive')

# 데이터셋 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/2025_zolzak/')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.7 MB/s eta 0:00:00


In [3]:
import torch
import pickle
import numpy as np
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from ultralytics import YOLO
from ultralytics.data.augment import LetterBox
from ultralytics.utils.instance import Instances
from PIL import Image
import time
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# ============= CUDA CLAHE 초기화 =============
try:
    clahe_cuda = cv2.cuda.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    USE_CUDA_CLAHE = True
    print("✓ CUDA CLAHE initialized successfully")
except Exception as e:
    print(f"✗ CUDA CLAHE initialization failed: {e}")
    print("  Falling back to CPU CLAHE")
    clahe_cpu = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    USE_CUDA_CLAHE = False

✗ CUDA CLAHE initialization failed: module 'cv2.cuda' has no attribute 'createCLAHE'
  Falling back to CPU CLAHE


In [5]:
# ============= 설정 =============
with open("id_stats.pkl", "rb") as f:
    stats = pickle.load(f)

mu_np, cov_inv_np, threshold = stats["mu"], stats["cov_inv"], stats["threshold"]

device = torch.device("cuda")
mu = torch.from_numpy(mu_np).float().to(device)
cov_inv = torch.from_numpy(cov_inv_np).float().to(device)

eval_val_img_dir = "dataset_prepared_05/eval/val/images/"
eval_val_lbl_dir = "dataset_prepared_05/eval/val/labels/"

model = YOLO("runs/train/dp05_yolov105/weights/best.pt")
model.to(device)
model.eval()

neck_layer = model.model.model[22]

In [6]:
# ============= CLAHE 적용 함수 =============
def apply_clahe_cuda(img_rgb):
    if USE_CUDA_CLAHE:
        img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)
        y_channel = img_ycrcb[:, :, 0]
        gpu_y = cv2.cuda_GpuMat()
        gpu_y.upload(y_channel)
        gpu_y_clahe = clahe_cuda.apply(gpu_y)
        y_clahe = gpu_y_clahe.download()
        img_ycrcb[:, :, 0] = y_clahe
        clahe_rgb = cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2RGB)
    else:
        img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)
        img_ycrcb[:, :, 0] = clahe_cpu.apply(img_ycrcb[:, :, 0])
        clahe_rgb = cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2RGB)
    return clahe_rgb

In [7]:
# ============= 배치 마할라노비스 거리 계산 =============
def batch_mahalanobis_distance(X, mu, cov_inv):
    diff = X - mu.unsqueeze(0)
    left = torch.matmul(diff, cov_inv)
    distances = torch.sqrt(torch.sum(left * diff, dim=1))
    return distances

class FeatureExtractor:
    def __init__(self, model, layer):
        self.features = None
        self.hook_handle = layer.register_forward_hook(self.hook_fn)

    def hook_fn(self, module, input, output):
        self.features = torch.mean(output, dim=(2, 3))

    def remove(self):
        self.hook_handle.remove()

In [12]:
# ============= OOD 경고 오버레이 생성 =============
# ============= 모델 imgsz와 원본 크기에 맞도록 조정 필요 =============
def create_ood_warning_overlay(img_shape, intensity=0.3, border_width=None):
    """
    네비게이션 스타일의 OOD 경고 오버레이 생성 (자동 크기 조정)
    """
    H, W = img_shape[:2]

    # 이미지 크기에 비례하는 테두리 두께
    if border_width is None:
        border_width = max(20, int(min(H, W) * 0.02))  # 이미지 크기의 2%

    overlay = np.zeros((H, W, 3), dtype=np.uint8)

    red_color = int(255 * intensity)
    overlay[:border_width, :] = [0, 0, red_color]
    overlay[-border_width:, :] = [0, 0, red_color]
    overlay[:, :border_width] = [0, 0, red_color]
    overlay[:, -border_width:] = [0, 0, red_color]

    corner_size = border_width * 2
    overlay[:corner_size, :corner_size] = [0, 0, 255]
    overlay[:corner_size, -corner_size:] = [0, 0, 255]
    overlay[-corner_size:, :corner_size] = [0, 0, 255]
    overlay[-corner_size:, -corner_size:] = [0, 0, 255]

    return overlay

def draw_ood_warning(img, ood_detected, alpha=0.5):
    """
    이미지에 OOD 경고 표시 (원본 크기 자동 조정)
    """
    if not ood_detected:
        return img

    overlay = create_ood_warning_overlay(img.shape, intensity=0.6)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)
    img_with_warning = cv2.addWeighted(img, 1, overlay, alpha, 0)

    # 텍스트 크기도 이미지에 비례하게
    H, W = img.shape[:2]
    font_scale = max(1.0, min(H, W) / 1000)  # 동적 폰트 크기
    thickness = max(2, int(font_scale * 2))

    font = cv2.FONT_HERSHEY_SIMPLEX
    text = "⚠ OOD DETECTED!"

    (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, thickness)

    x = (W - text_w) // 2
    y = int(H * 0.08)  # 상단 8% 위치

    # 텍스트 배경
    padding = 15
    cv2.rectangle(img_with_warning,
                  (x - padding, y - text_h - padding),
                  (x + text_w + padding, y + padding),
                  (0, 0, 0), -1)

    # 텍스트
    cv2.putText(img_with_warning, text, (x, y),
                font, font_scale, (0, 0, 255), thickness)

    return img_with_warning

# def create_ood_warning_overlay(img_shape, intensity=0.3, border_width=30):
#     """
#     네비게이션 스타일의 OOD 경고 오버레이 생성
#     Args:
#         img_shape: (H, W, C)
#         intensity: 빨간색 강도 (0~1)
#         border_width: 테두리 두께 (픽셀)
#     Returns:
#         overlay: (H, W, 3) uint8 배열
#     """
#     H, W = img_shape[:2]
#     overlay = np.zeros((H, W, 3), dtype=np.uint8)

#     # 빨간색 테두리 (상하좌우)
#     red_color = int(255 * intensity)
#     overlay[:border_width, :] = [red_color, 0, 0]  # 상단
#     overlay[-border_width:, :] = [red_color, 0, 0]  # 하단
#     overlay[:, :border_width] = [red_color, 0, 0]  # 좌측
#     overlay[:, -border_width:] = [red_color, 0, 0]  # 우측

#     # 코너 강조 (선택적)
#     corner_size = border_width * 2
#     overlay[:corner_size, :corner_size] = [255, 0, 0]  # 좌상
#     overlay[:corner_size, -corner_size:] = [255, 0, 0]  # 우상
#     overlay[-corner_size:, :corner_size] = [255, 0, 0]  # 좌하
#     overlay[-corner_size:, -corner_size:] = [255, 0, 0]  # 우하

#     return overlay

# def draw_ood_warning(img, ood_detected, alpha=0.5):
#     """
#     이미지에 OOD 경고 표시
#     Args:
#         img: (H, W, 3) BGR 이미지
#         ood_detected: bool, OOD 검출 여부
#         alpha: 오버레이 투명도
#     Returns:
#         img_with_warning: (H, W, 3) BGR 이미지
#     """
#     if not ood_detected:
#         return img

#     overlay = create_ood_warning_overlay(img.shape, intensity=0.6, border_width=20)
#     # RGB -> BGR 변환
#     overlay = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)

#     # 블렌딩
#     img_with_warning = cv2.addWeighted(img, 1, overlay, alpha, 0)

#     # "OOD DETECTED!" 텍스트 추가
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     text = "⚠ OOD DETECTED!"
#     font_scale = 1.2
#     thickness = 3

#     # 텍스트 크기 계산
#     (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, thickness)

#     # 중앙 상단에 배치
#     x = (img.shape[1] - text_w) // 2
#     y = 50

#     # 텍스트 배경 (검은색 반투명)
#     cv2.rectangle(img_with_warning,
#                   (x-10, y-text_h-10),
#                   (x+text_w+10, y+10),
#                   (0, 0, 0), -1)

#     # 텍스트 (빨간색)
#     cv2.putText(img_with_warning, text, (x, y),
#                 font, font_scale, (0, 0, 255), thickness)

#     return img_with_warning

# def draw_bboxes(img, results, conf_threshold=0.001):
#     """
#     YOLO 결과에서 ID 클래스 bbox 그리기
#     Args:
#         img: (H, W, 3) BGR 이미지
#         results: YOLO results 객체
#         conf_threshold: confidence threshold
#     Returns:
#         img_with_boxes: (H, W, 3) BGR 이미지
#     """
#     img_with_boxes = img.copy()

#     if results and len(results) > 0:
#         result = results[0]  # 첫 번째 이미지 결과

#         if result.boxes is not None and len(result.boxes) > 0:
#             boxes = result.boxes.cpu().numpy()

#             for box in boxes:
#                 conf = box.conf[0]
#                 if conf < conf_threshold:
#                     continue

#                 cls_id = int(box.cls[0])
#                 x1, y1, x2, y2 = box.xyxy[0].astype(int)

#                 # bbox 그리기 (녹색)
#                 cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)

#                 # 라벨 배경
#                 label = f"Class {cls_id}: {conf:.2f}"
#                 (label_w, label_h), _ = cv2.getTextSize(
#                     label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
#                 )
#                 cv2.rectangle(img_with_boxes,
#                              (x1, y1-label_h-10),
#                              (x1+label_w, y1),
#                              (0, 255, 0), -1)

#                 # 라벨 텍스트
#                 cv2.putText(img_with_boxes, label,
#                            (x1, y1-5),
#                            cv2.FONT_HERSHEY_SIMPLEX,
#                            0.5, (0, 0, 0), 1)

#     return img_with_boxes

def draw_bboxes(img, results, original_shape, model_input_shape=(544, 960), conf_threshold=0.001):
    """
    YOLO 결과에서 ID 클래스 bbox 그리기 (좌표 스케일 보정 포함)
    Args:
        img: (H, W, 3) BGR 이미지 (원본 크기)
        results: YOLO results 객체
        original_shape: (H, W) 원본 이미지 크기
        model_input_shape: (H, W) 모델 입력 크기 (544, 960)
        conf_threshold: confidence threshold
    Returns:
        img_with_boxes: (H, W, 3) BGR 이미지
    """
    img_with_boxes = img.copy()
    orig_h, orig_w = original_shape
    model_h, model_w = model_input_shape

    # LetterBox 적용 시 스케일 계산
    # LetterBox는 aspect ratio를 유지하면서 리사이즈
    scale = min(model_h / orig_h, model_w / orig_w)

    # 실제 리사이즈된 크기
    new_h = int(orig_h * scale)
    new_w = int(orig_w * scale)

    # 패딩 계산
    pad_h = (model_h - new_h) / 2
    pad_w = (model_w - new_w) / 2

    if results and len(results) > 0:
        result = results[0]  # 첫 번째 이미지 결과

        if result.boxes is not None and len(result.boxes) > 0:
            boxes = result.boxes.cpu().numpy()

            for box in boxes:
                conf = box.conf[0]
                if conf < conf_threshold:
                    continue

                cls_id = int(box.cls[0])

                # 모델 출력 좌표 (960x544 기준)
                x1_model, y1_model, x2_model, y2_model = box.xyxy[0]

                # 패딩 제거
                x1_unpad = x1_model - pad_w
                y1_unpad = y1_model - pad_h
                x2_unpad = x2_model - pad_w
                y2_unpad = y2_model - pad_h

                # 원본 크기로 스케일 복원
                x1_orig = int(x1_unpad / scale)
                y1_orig = int(y1_unpad / scale)
                x2_orig = int(x2_unpad / scale)
                y2_orig = int(y2_unpad / scale)

                # 이미지 경계 내로 클리핑
                x1_orig = max(0, min(x1_orig, orig_w))
                y1_orig = max(0, min(y1_orig, orig_h))
                x2_orig = max(0, min(x2_orig, orig_w))
                y2_orig = max(0, min(y2_orig, orig_h))

                # bbox 그리기 (녹색)
                cv2.rectangle(img_with_boxes,
                             (x1_orig, y1_orig), (x2_orig, y2_orig),
                             (0, 255, 0), 3)  # 두께 2->3 (큰 이미지용)

                # 라벨 배경
                label = f"Class {cls_id}: {conf:.2f}"
                font_scale = 0.8  # 0.5 -> 0.8 (큰 이미지용)
                thickness = 2
                (label_w, label_h), _ = cv2.getTextSize(
                    label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness
                )

                # 라벨이 이미지 밖으로 나가지 않도록
                label_y = max(label_h + 10, y1_orig)

                cv2.rectangle(img_with_boxes,
                             (x1_orig, label_y - label_h - 10),
                             (x1_orig + label_w + 10, label_y),
                             (0, 255, 0), -1)

                # 라벨 텍스트
                cv2.putText(img_with_boxes, label,
                           (x1_orig + 5, label_y - 5),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           font_scale, (0, 0, 0), thickness)

    return img_with_boxes

In [13]:
# ============= 데이터셋 =============
def load_gt_labels(label_dir):
    gt_map = {}
    for txt_file in Path(label_dir).glob("*.txt"):
        with open(txt_file, "r", encoding="utf-8") as f:
            lines = f.readlines()
            fname = txt_file.stem
            gt_map[fname] = [int(line.split()[0]) for line in lines if line.strip()]
    return gt_map

class OODDatasetWithCLAHE(Dataset):
    def __init__(self, img_dir, label_dir, gt_labels, imgsz=960, use_clahe=True):
        self.img_dir = Path(img_dir)
        self.label_dir = Path(label_dir) if label_dir else None
        self.gt_labels = gt_labels
        self.imgsz = imgsz
        self.use_clahe = use_clahe

        # 모든 이미지 (OOD만이 아닌 전체)
        self.img_paths = []
        for p in sorted(self.img_dir.iterdir()):
            if p.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                self.img_paths.append(str(p))

        print(f"Found {len(self.img_paths)} images")
        print(f"CLAHE preprocessing: {'Enabled' if use_clahe else 'Disabled'}")

        self.letterbox = LetterBox(new_shape=(544, 960), auto=False)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        stem = Path(img_path).stem

        # 원본 이미지 로드 (시각화용)
        original_img = cv2.imread(img_path)

        img = Image.open(img_path).convert("RGB")
        img_array = np.array(img)

        # CLAHE 적용
        if self.use_clahe:
            img_array = apply_clahe_cuda(img_array)

        # 라벨 로드
        if self.label_dir and (self.label_dir / f"{stem}.txt").exists():
            with open(self.label_dir / f"{stem}.txt") as f:
                lines = [list(map(float, line.strip().split()))
                        for line in f if line.strip()]
            if lines:
                cls, bboxes = zip(*[(int(l[0]), l[1:]) for l in lines])
                bboxes = np.array(bboxes, dtype=np.float32)
                cls = np.array(cls)
            else:
                bboxes = np.zeros((0, 4), dtype=np.float32)
                cls = np.array([])
        else:
            bboxes = np.zeros((0, 4), dtype=np.float32)
            cls = np.array([])

        inst = Instances(
            bboxes=bboxes,
            segments=np.zeros((len(bboxes), 0, 2), dtype=np.float32),
            bbox_format="xywh"
        )

        sample = {
            "img": img_array,
            "cls": cls,
            "instances": inst,
            "ori_shape": img_array.shape[:2],
            "resized_shape": img_array.shape[:2],
            "path": str(img_path),
            "batch_idx": 0,
            "original_img": original_img  # 시각화용 원본 추가
        }

        sample = self.letterbox(sample)
        img_tensor = torch.from_numpy(sample["img"]).permute(2, 0, 1).contiguous()
        sample["img"] = img_tensor

        return sample

def custom_collate_fn(batch):
    collated = {}
    for key in batch[0].keys():
        if key == "instances":
            collated[key] = [item[key] for item in batch]
        elif key in ["img"]:
            collated[key] = torch.stack([item[key] for item in batch])
        elif key in ["original_img"]:
            collated[key] = [item[key] for item in batch]
        else:
            collated[key] = [item[key] for item in batch]
    return collated

In [ ]:
# # ============= 통합 추론 및 시각화 파이프라인 =============
# def inference_with_visualization(model, dataloader, mu, cov_inv, threshold,
#                                  device, save_dir="output_visualizations"):
#     """
#     OOD 검출 + ID bbox 그리기 + 시각화 통합 파이프라인
#     """
#     save_dir = Path(save_dir)
#     save_dir.mkdir(exist_ok=True, parents=True)

#     feature_extractor = FeatureExtractor(model, neck_layer)

#     all_scores = []
#     all_results = []  # YOLO detection 결과 저장
#     all_original_imgs = []
#     all_paths = []

#     model.eval()

#     with torch.no_grad():
#         for batch_idx, batch in enumerate(tqdm(dataloader, desc="Processing")):
#             imgs = batch["img"].to(device, non_blocking=True).float() / 255.0
#             original_imgs = batch["original_img"]
#             paths = batch["path"]

#             # YOLO 추론 (bbox 검출)
#             results = model(imgs, conf=0.001, verbose=False)

#             # Feature 추출 (OOD 검출용)
#             if feature_extractor.features is not None:
#                 feats = feature_extractor.features
#             else:
#                 feat_map = model.model.model[:23](imgs)
#                 feats = torch.mean(feat_map, dim=(2, 3))

#             # 마할라노비스 거리 계산
#             distances = batch_mahalanobis_distance(feats, mu, cov_inv)

#             all_scores.append(distances)
#             all_results.extend(results)
#             all_original_imgs.extend(original_imgs)
#             all_paths.extend(paths)

#     feature_extractor.remove()

#     # 모든 스코어 결합
#     all_scores = torch.cat(all_scores, dim=0).cpu().numpy()
#     ood_predictions = all_scores > threshold

#     # 시각화
#     print("\nGenerating visualizations...")
#     for idx, (img, score, is_ood, result, path) in enumerate(
#         tqdm(zip(all_original_imgs, all_scores, ood_predictions, all_results, all_paths),
#              total=len(all_original_imgs), desc="Saving")
#     ):
#         # 1. ID 클래스 bbox 그리기
#         img_with_boxes = draw_bboxes(img, [result], conf_threshold=0.001)

#         # 2. OOD 경고 표시
#         img_final = draw_ood_warning(img_with_boxes, is_ood, alpha=0.5)

#         # 3. 스코어 정보 표시 (우하단)
#         info_text = f"Score: {score:.2f} | Threshold: {threshold:.2f}"
#         status_text = "OOD" if is_ood else "ID"
#         status_color = (0, 0, 255) if is_ood else (0, 255, 0)

#         cv2.putText(img_final, info_text, (10, img_final.shape[0]-40),
#                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
#         cv2.putText(img_final, f"Status: {status_text}", (10, img_final.shape[0]-10),
#                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, status_color, 2)

#         # 4. 저장
#         filename = Path(path).stem
#         save_path = save_dir / f"{filename}_result.jpg"
#         cv2.imwrite(str(save_path), img_final)

#     return all_scores, ood_predictions

In [14]:
def inference_with_visualization(model, dataloader, mu, cov_inv, threshold,
                                 device, save_dir="output_visualizations",
                                 model_input_shape=(544, 960)):
    """
    OOD 검출 + ID bbox 그리기 + 시각화 통합 파이프라인
    """
    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)

    feature_extractor = FeatureExtractor(model, neck_layer)

    all_scores = []
    all_results = []
    all_original_imgs = []
    all_paths = []

    model.eval()

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader, desc="Processing")):
            imgs = batch["img"].to(device, non_blocking=True).float() / 255.0
            original_imgs = batch["original_img"]
            paths = batch["path"]

            # YOLO 추론
            results = model(imgs, conf=0.001, verbose=False)

            # Feature 추출
            if feature_extractor.features is not None:
                feats = feature_extractor.features
            else:
                feat_map = model.model.model[:23](imgs)
                feats = torch.mean(feat_map, dim=(2, 3))

            distances = batch_mahalanobis_distance(feats, mu, cov_inv)

            all_scores.append(distances)
            all_results.extend(results)
            all_original_imgs.extend(original_imgs)
            all_paths.extend(paths)

    feature_extractor.remove()

    all_scores = torch.cat(all_scores, dim=0).cpu().numpy()
    ood_predictions = all_scores > threshold

    # 시각화
    print("\nGenerating visualizations...")
    for idx, (img, score, is_ood, result, path) in enumerate(
        tqdm(zip(all_original_imgs, all_scores, ood_predictions, all_results, all_paths),
             total=len(all_original_imgs), desc="Saving")
    ):
        orig_h, orig_w = img.shape[:2]

        # 1. ID 클래스 bbox 그리기 (스케일 보정 포함)
        img_with_boxes = draw_bboxes(
            img, [result],
            original_shape=(orig_h, orig_w),
            model_input_shape=model_input_shape,
            conf_threshold=0.001
        )

        # 2. OOD 경고 표시
        img_final = draw_ood_warning(img_with_boxes, is_ood, alpha=0.5)

        # 3. 스코어 정보 (크기에 맞게 조정)
        font_scale = max(0.6, min(orig_h, orig_w) / 1500)
        thickness = max(2, int(font_scale * 3))

        info_text = f"Score: {score:.2f} | Threshold: {threshold:.2f}"
        status_text = "OOD" if is_ood else "ID"
        status_color = (0, 0, 255) if is_ood else (0, 255, 0)

        y_offset = orig_h - int(orig_h * 0.05)
        cv2.putText(img_final, info_text, (20, y_offset - 40),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)
        cv2.putText(img_final, f"Status: {status_text}", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 1.2, status_color, thickness)

        # 4. 저장
        filename = Path(path).stem
        save_path = save_dir / f"{filename}_result.jpg"
        cv2.imwrite(str(save_path), img_final)

    return all_scores, ood_predictions

In [ ]:
# ============= 실행 =============
eval_val_gt_labels = load_gt_labels(eval_val_lbl_dir)

fps_test_loader = DataLoader(
    OODDatasetWithCLAHE(
        eval_val_img_dir,
        eval_val_lbl_dir,
        eval_val_gt_labels,
        use_clahe=True
    ),
    batch_size=16,
    shuffle=False,
    collate_fn=custom_collate_fn,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

num_imgs = len(fps_test_loader.dataset)

# Warmup (첫 실행 오버헤드 제거)
print("Warming up...")
_ = inference_with_visualization(
    model, fps_test_loader, mu, cov_inv, threshold,
    device, save_dir="output_visualizations_warmup"
)

Found 5247 images
CLAHE preprocessing: Enabled
Warming up...


Processing: 100%|██████████| 328/328 [02:39<00:00,  2.05it/s]



Generating visualizations...


Saving:  32%|███▏      | 1696/5247 [07:31<40:48,  1.45it/s]

In [ ]:
# 실제 측정
print("\n\nActual measurement with visualization...")
start = time.time()
pred_scores, pred_labels = inference_with_visualization(
    model, fps_test_loader, mu, cov_inv, threshold,
    device, save_dir="output_visualizations_final"
)
end = time.time()

inference_time = end - start
fps = num_imgs / inference_time

print(f"\n{'='*60}")
print(f"Total Images: {num_imgs}")
print(f"Inference + Visualization Time: {inference_time:.2f} seconds")
print(f"FPS: {fps:.2f} frames per second")
print(f"OOD Detected: {pred_labels.sum()} / {num_imgs}")
print(f"Output saved to: output_visualizations_final/")
print(f"{'='*60}")



Actual measurement with visualization...


Processing: 100%|██████████| 328/328 [01:42<00:00,  3.21it/s]



Generating visualizations...


Saving: 100%|██████████| 5247/5247 [02:04<00:00, 42.19it/s]


Total Images: 5247
Inference + Visualization Time: 226.73 seconds
FPS: 23.14 frames per second
OOD Detected: 5235 / 5247
Output saved to: output_visualizations_final/


In [ ]:
# CLAHE 비교 분석

fps_test_loader = DataLoader(
    OODDatasetWithCLAHE(
        eval_val_img_dir,
        eval_val_lbl_dir,
        eval_val_gt_labels,
        use_clahe=False
    ),
    batch_size=16,
    shuffle=False,
    collate_fn=custom_collate_fn,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

Found 5247 images
CLAHE preprocessing: Disabled


In [ ]:
print("\n\nActual measurement with visualization...")
start = time.time()
pred_scores, pred_labels = inference_with_visualization(
    model, fps_test_loader, mu, cov_inv, threshold,
    device, save_dir="output_visualizations_final_wo_clahe"
)
end = time.time()

inference_time = end - start
fps = num_imgs / inference_time

print(f"\n{'='*60}")
print(f"Total Images: {num_imgs}")
print(f"Inference + Visualization Time: {inference_time:.2f} seconds")
print(f"FPS: {fps:.2f} frames per second")
print(f"OOD Detected: {pred_labels.sum()} / {num_imgs}")
print(f"Output saved to: output_visualizations_final_wo_clahe/")
print(f"{'='*60}")



Actual measurement with visualization...


Processing: 100%|██████████| 328/328 [01:49<00:00,  3.01it/s]



Generating visualizations...


Saving: 100%|██████████| 5247/5247 [01:58<00:00, 44.42it/s]


Total Images: 5247
Inference + Visualization Time: 227.29 seconds
FPS: 23.09 frames per second
OOD Detected: 4831 / 5247
Output saved to: output_visualizations_final_wo_clahe/
